In [55]:
import requests
import json
import pandas as pd

In [56]:
limit = 10000000

In [57]:
url = f'https://data.cdc.gov/resource/u6jv-9ijr.json?$limit={limit}'

In [58]:
data = requests.get(url)
data

<Response [200]>

In [59]:
data = data.json()

In [60]:
df = pd.DataFrame(data)
df

,jurisdiction,week_ending_date,state_abbreviation,mmwryear,mmwrweek,cause_group,number_of_deaths,cause_subgroup,time_period,average_number_of_deaths,type,difference_from_2015_2019_to_2020,percent_difference_from_15_19_to_20,note
0,New Jersey,2018-02-03T00:00:00.000,NJ,2018,5,Other select causes,49,Sepsis,2015-2019,42,Predicted (weighted),NaN,NaN,NaN
1,Alabama,2015-01-10T00:00:00.000,AL,2015,1,Other select causes,22,Sepsis,2015-2019,24,Predicted (weighted),NaN,NaN,NaN
2,Alabama,2015-01-10T00:00:00.000,AL,2015,1,Other select causes,22,Sepsis,2015-2019,24,Unweighted,NaN,NaN,NaN
3,Alabama,2016-01-09T00:00:00.000,AL,2016,1,Other select causes,13,Sepsis,2015-2019,24,Predicted (weighted),NaN,NaN,NaN
4,Alabama,2016-01-09T00:00:00.000,AL,2016,1,Other select causes,13,Sepsis,2015-2019,24,Unweighted,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481135,Oregon,2023-04-22T00:00:00.000,OR,2023,16,Alzheimer disease and dementia,77,Alzheimer disease and dementia,2023,77,Predicted (weighted),-2,-2.5,Data in recent weeks are incomplete. Only 60% ...
481136,Oregon,2022-04-23T00:00:00.000,OR,2022,16,Alzheimer disease and dementia,98,Alzheimer disease and dementia,2022,98,Unweighted,NaN,NaN,Data in recent weeks are incomplete. Only 60% ...
481137,Oregon,2022-04-23T00:00:00.000,OR,2022,16,Alzheimer disease and dementia,98,Alzheimer disease and dementia,2022,98,Predicted (weighted),19,24.1,Data in recent weeks are incomplete. Only 60% ...
481138,Oregon,2021-04-24T00:00:00.000,OR,2021,16,Alzheimer disease and dementia,79,Alzheimer disease and dementia,2021,79,Unweighted,NaN,NaN,NaN


In [61]:
df['time_period'].unique()

array(['2015-2019', '2020', '2021', '2022', '2023'], dtype=object)

In [62]:
df['cause_group'].unique()

array(['Other select causes', 'Respiratory diseases',
       'Circulatory diseases', 'Malignant neoplasms',
       'Alzheimer disease and dementia'], dtype=object)

In [63]:
df = df.loc[df['type'] == "Unweighted"]

In [64]:
respiratory_df = df.loc[df['cause_subgroup'] == 'Influenza and pneumonia']
respiratory_df

,jurisdiction,week_ending_date,state_abbreviation,mmwryear,mmwrweek,cause_group,number_of_deaths,cause_subgroup,time_period,average_number_of_deaths,type,difference_from_2015_2019_to_2020,percent_difference_from_15_19_to_20,note
221053,Alabama,2015-01-10T00:00:00.000,AL,2015,1,Respiratory diseases,48,Influenza and pneumonia,2015-2019,34,Unweighted,NaN,NaN,NaN
221055,Alabama,2016-01-09T00:00:00.000,AL,2016,1,Respiratory diseases,26,Influenza and pneumonia,2015-2019,34,Unweighted,NaN,NaN,NaN
221057,Alabama,2017-01-07T00:00:00.000,AL,2017,1,Respiratory diseases,29,Influenza and pneumonia,2015-2019,34,Unweighted,NaN,NaN,NaN
221059,Alabama,2018-01-06T00:00:00.000,AL,2018,1,Respiratory diseases,44,Influenza and pneumonia,2015-2019,34,Unweighted,NaN,NaN,NaN
221061,Alabama,2019-01-05T00:00:00.000,AL,2019,1,Respiratory diseases,21,Influenza and pneumonia,2015-2019,34,Unweighted,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248211,Wisconsin,2016-12-31T00:00:00.000,WI,2016,52,Respiratory diseases,16,Influenza and pneumonia,2015-2019,16,Unweighted,NaN,NaN,NaN
248213,Wisconsin,2017-12-30T00:00:00.000,WI,2017,52,Respiratory diseases,19,Influenza and pneumonia,2015-2019,16,Unweighted,NaN,NaN,NaN
248215,Wisconsin,2018-12-29T00:00:00.000,WI,2018,52,Respiratory diseases,16,Influenza and pneumonia,2015-2019,16,Unweighted,NaN,NaN,NaN
248217,Wisconsin,2019-12-28T00:00:00.000,WI,2019,52,Respiratory diseases,11,Influenza and pneumonia,2015-2019,16,Unweighted,NaN,NaN,NaN


In [65]:
df.columns

Index(['jurisdiction', 'week_ending_date', 'state_abbreviation', 'mmwryear',
       'mmwrweek', 'cause_group', 'number_of_deaths', 'cause_subgroup',
       'time_period', 'average_number_of_deaths', 'type',
       'difference_from_2015_2019_to_2020',
       'percent_difference_from_15_19_to_20', 'note'],
      dtype='object')

In [66]:
df['type'].unique()

array(['Unweighted'], dtype=object)

I want this to be a dataset of actual, rather than predicted so let's filter for that. 

In [67]:
respiratory_df = respiratory_df[['week_ending_date', 'number_of_deaths','average_number_of_deaths' ]]

In [68]:
respiratory_df['week_ending_date'] = pd.to_datetime(respiratory_df['week_ending_date'])

C:\Users\risab\AppData\Local\Temp\ipykernel_48108\906734675.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  respiratory_df['week_ending_date'] = pd.to_datetime(respiratory_df['week_ending_date'])


In [69]:
respiratory_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13584 entries, 221053 to 248219
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   week_ending_date          13584 non-null  datetime64[ns]
 1   number_of_deaths          13584 non-null  object        
 2   average_number_of_deaths  13584 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 424.5+ KB


In [70]:
respiratory_df = respiratory_df.set_index("week_ending_date")

,number_of_deaths,average_number_of_deaths
week_ending_date,,
2015-01-10,48,34
2016-01-09,26,34
2017-01-07,29,34
2018-01-06,44,34
2019-01-05,21,34
...,...,...
2016-12-31,16,16
2017-12-30,19,16
2018-12-29,16,16


In [72]:
respiratory_df = respiratory_df.sort_index()
respiratory_df

,number_of_deaths,average_number_of_deaths
week_ending_date,,
2015-01-10,48,34
2015-01-10,119,112
2015-01-10,54,51
2015-01-10,123,72
2015-01-10,15,14
...,...,...
2023-09-09,15,15
2023-09-09,13,13
2023-09-09,34,34
